In [1]:
import random
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage
import sklearn
from sklearn.model_selection import train_test_split
import sktime
from sktime.datatypes._panel._convert import from_3d_numpy_to_nested
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report


In [2]:
def maximum(raw, box_size, mode='nearest'): # Maximum Filter
    raw_maximum=ndimage.maximum_filter(raw,box_size,mode=mode)
    return raw_maximum.real

def minimum(raw, box_size, mode='nearest'): # Maximum Filter
    raw_minimum=ndimage.minimum_filter(raw,box_size,mode=mode)
    return raw_minimum.real

def denoise_fft(data, ifftn): # Fast Fourier Transformation
    fft_signal = np.fft.fft(data)
    
    # Reconstruct the original signal
    fft_signal[ifftn:len(fft_signal)//2]=0
    fft_signal[len(fft_signal)//2:-ifftn]=0
    reconstructed_signal = np.fft.ifft(fft_signal)
    
    return reconstructed_signal.real

def smooth(x,beta): # Kaiser Window Smoothing
    window_len=11  # extending the data at beginning and at the end to apply the window at the borders

    s = np.r_[x[window_len-1:0:-1],x,x[-1:-window_len:-1]]
    w = np.kaiser(window_len,beta)
    y = np.convolve(w/w.sum(),s,mode='valid')
    return y[5:len(y)-5]


In [31]:
from joblib import dump, load
model = load('model777.joblib')


In [34]:
import serial
import numpy as np 
import pandas as pd
from collections import Counter

ser = serial.Serial('COM3', 115200, timeout=1)
skip = 1
data_batch = []
count = 0
# 데이터 읽기를 시도하는 루프
try:
    while True:
        if ser.in_waiting > 0:
            data = ser.readline()
            # ISO-8859-1
            decoded_data = data.decode('ISO-8859-1').rstrip()

            if skip == 0:
                try:
                    int_data = [int(x) for x in decoded_data.split(',')]
                    data_batch.append(int_data)
                except ValueError:
                    print("데이터 변환 중 오류 발생:", decoded_data)

                # 190행이 모였는지 확인
                if len(data_batch) == 190:
                    # 리스트를 DataFrame으로 변환
                    df = pd.DataFrame(data_batch, columns=['a0', 'a1', 'a2', 'a3', 'a4', 'a5'])
                    depths_list = []
                    for i in range(6):
                        column = f'a{i}'  # 필터링된 데이터의 각 채널 컬럼명
                        depths = smooth(denoise_fft(maximum(df[column], 5)-minimum(df[column], 5), 5), 5)
                        depths_list.append(depths)

                    df_2d = pd.DataFrame(depths_list)

                    df_2d_cleaned = df_2d.dropna(axis=1, how='any')

                    result = df_2d_cleaned.to_numpy().flatten()

                    #np.set_printoptions(precision=2, suppress=True)
                    # 모델 예측 확률
                    probabilities = model.predict_proba(result)

                    # 임계값 설정 (예: 70% 이상일 때만 예측 결과를 사용)
                    threshold = 0.8
                    # 각 샘플의 확률을 순회
                    
                        # 각 클래스에 대한 예측 확률 중 가장 높은 값을 가져옵니다.
                    max_prob = np.max(probabilities)  # 가장 높은 확률 값을 구합니다.
                    max_arg = np.argmax(probabilities)

                    # 가장 높은 확률 값이 threshold 이상인지 확인
                    if max_prob >= threshold:
                        count = max_arg
                            
                    if count == 0:
                        print("rest")
                    elif count == 1:
                        print("rock")
                    elif count == 2:
                        print("scissors")
                    elif count == 3:
                        print("right")
                    elif count == 4:
                        print("left")
                    elif count == 5:
                        print("fire")
                    elif count == 6:
                        print("paper")
                    elif count == 7:
                        print("paper_left")
                    elif count == 8:
                        print("paper_right")
                        

                    data_batch = data_batch[20:]
            skip = 0

except KeyboardInterrupt:
    print('프로그램을 종료합니다.')
    ser.close()  # 시리얼 포트 닫기

paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rock
rock
rock
rock
rock
rock
rock
rock
rock
rock
rock
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rest 0
rock
rock
rock
rock
rock
rock
rock
rock
paper_right
paper_right
paper_right
paper_right
paper_right
paper_right
paper_right
paper
paper
paper
paper
paper
paper
rock
rock
rock
rock
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
프로그램을 종료합니다.


In [7]:
import socket
import serial
import time
import numpy as np 
import pandas as pd

ser = serial.Serial('COM3', 115200, timeout=1)
skip = 1
data_batch = []
# 데이터 읽기를 시도하는 루프
try:
    while True:
        if ser.in_waiting > 0:
            data = ser.readline()
            # ISO-8859-1
            decoded_data = data.decode('ISO-8859-1').rstrip()

            if skip == 0:
                try:
                    int_data = [int(x) for x in decoded_data.split(',')]
                    data_batch.append(int_data)
                except ValueError:
                    print("데이터 변환 중 오류 발생:", decoded_data)

                # 190행이 모였는지 확인
                if len(data_batch) == 190:
                    # 리스트를 DataFrame으로 변환
                    df = pd.DataFrame(data_batch, columns=['a0', 'a1', 'a2', 'a3', 'a4', 'a5'])
                    depths_list = []
                    for i in range(6):
                        column = f'a{i}'  # 필터링된 데이터의 각 채널 컬럼명
                        depths = smooth(denoise_fft(maximum(df[column], 5)-minimum(df[column], 5), 5), 5)
                        diff = np.diff(depths)
                        depths_list.append(depths)
                        depths_list.append(diff)
                    df_2d = pd.DataFrame(np.hstack(depths_list))

                    df_2d_cleaned = df_2d.dropna(axis=1, how='any')

                    result = df_2d_cleaned.to_numpy().flatten()

                    predicted = model.predict(result)[0]


                    if predicted == 0:
                        print("rest")
                    elif predicted == 1:
                        print("rock")
                    elif predicted == 2:
                        print("scissors")
                    elif predicted == 3:
                        print("right")
                    elif predicted == 4:
                        print("left")
                    elif predicted == 5:
                        print("fire")
                    elif predicted == 6:
                        print("paper")
                    elif predicted == 7:
                        print("paper_left")
                    elif predicted == 8:
                        print("paper_right")
                    # elif model.predict(result)[0] == 2:
                    #     print("bird_gun")

                    data_batch = data_batch[20:]
            skip = 0

except KeyboardInterrupt:
    print('프로그램을 종료합니다.')
    ser.close()  # 시리얼 포트 닫기

paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
scissors
paper_left
scissors
scissors
rest
rest
rest
scissors
paper_left
paper
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper
paper
paper
paper
paper
paper
paper
paper
paper
paper
paper
paper
rock
rock
rock
paper_left
paper_left
paper_left
rock
rock
paper
paper
paper
paper
paper
paper
paper
paper
paper
paper
rock
rock
rock
rock
rock
rock
rock
rock
rock
rock
rock
rock
rock
rock
rock
rock
rock
rock
paper_left
paper_left
rock
rock
rock
paper
paper
paper
paper_left
paper
paper_left
paper
paper
paper
paper
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper_left
paper
paper
paper
paper
paper
paper
paper_left
paper
paper_left
paper_left
paper_left
paper_left
paper_left
paper_le

In [ ]:
import socket
import time

def connect_to_unity():
    sock = None
    while sock is None:
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.connect(('127.0.0.1', 5005))  # 유니티 서버 IP와 포트
            print("Connected to Unity")
        except socket.error:
            print("Connection failed, retrying in 5 seconds...")
            time.sleep(5)
            sock = None
    return sock

# 자동 재연결을 시도
sock = connect_to_unity()

while True:
    try:
        # 메시지 전송
        message = "EMG Data"
        sock.sendall(message.encode())
        
        # 유니티에서 응답 수신
        response = sock.recv(1024)
        print(f"Received: {response.decode()}")
    except socket.error:
        print("Connection lost, reconnecting...")
        sock.close()
        sock = connect_to_unity()
